In [10]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from scipy.stats import distributions as iid
from matplotlib import pyplot as plt

In [19]:
# I'm pretty sure this part is right.
def HFUL(y, X, Z):
    n = y.shape[0]
    k = X.shape[1]
    P = Z @ np.linalg.inv(Z.T @ Z) @ Z.T
    X_bar = np.column_stack((y, X))
    # Calculate alpha hat
    PXbarXbar = np.zeros((X_bar.shape[1], X_bar.shape[1]))
    for i in range(n):
        PXbarXbar = PXbarXbar + P[i, i] * np.outer(X_bar[i, :], X_bar[i, :])
    A = np.linalg.inv(X_bar.T @ X_bar) @ (X_bar.T @ P @ X_bar - PXbarXbar)
    w, v = np.linalg.eig(A)
    alpha_tilde = np.amin(w)
    alpha_hat = ((n+1) * alpha_tilde - 1)/(n + alpha_tilde - 1)
    # Now calculate b HFUL
    PXX = np.zeros((X.shape[1], X.shape[1]))
    PXy = np.zeros(k)
    for i in range(n):
        PXX = PXX + P[i, i] * np.outer(X[i, :], X[i, :])
    for i in range(n):
        PXy = PXy + P[i, i] * X[i, :] * y[i]
    b_hful_term1 = X.T @ P @ X - PXX - alpha_hat * X.T @ X
    b_hful_term2 = X.T @ P @ y - PXy - alpha_hat * X.T @ y
    return np.linalg.inv(b_hful_term1) @ b_hful_term2

In [ ]:
""" OLD DGP
I know this part has problems. The covariance matrix must be positive definite. I don't want to have to specify every
term of the covariance matrix, so I just let it be a random pos def matrix (using make_spd_matrix). But then I changed 
some of the terms to make sure that my z variables are uncorrelated with the error term u. After doing this, the matrix
is only sometimes positive definite. When an error pops up, I just keep retrying until I get lucky and it's still
positive definite.

# Generate data, use random draws from multivariate normal
n = 10000
k = 3
l = 50
dim_cov_matrix = k + l + 1
cov_matrix = make_spd_matrix(dim_cov_matrix) # initialize covariance matrix as random
for i in range(k, l):
    cov_matrix[i, (k+l)] = 0 # make sure exclusion restriction holds
    cov_matrix[(k+l), i] = 0
data_dist = multivariate_normal(cov=cov_matrix)
data = data_dist.rvs(n)
X = data[:, range(0, k)]
Z = data[:, range(k, l)]
u = data[:, k+l]
"""

In [17]:
"""This code is for k=1, l arbitrary. Make sure Z*pi or Z@pi is defined."""

def linear_dgp(N, beta, l, pi, var_u, var_v, cov_uv):
    uv_dist = multivariate_normal(cov=[[var_u, cov_uv], [cov_uv, var_v]]) # this allows u and v to be correlated
    uv_data = uv_dist.rvs(N)
    u = uv_data[:, 0]
    v = uv_data[:, 1]
    Z = iid.norm().rvs(N) # initialize our Z matrix as Nx1. Note z is independent of u and v.
    for _ in range(l-1):
        Z = np.column_stack((Z, iid.norm().rvs(N))) # If we want multiple instruments, add columns to Z
    if l == 1:
        X = Z*pi + v
    else:
        X = Z@pi + v
    y = X*beta + u # Since X and u are correlated, OLS should be bad.

    df = pd.DataFrame({'y':y,'x':X,'z':Z,'Constant':1})

    return df[['y']],df[['Constant','x']],df[['Constant','z']]

In [22]:
y,X,Z = linear_dgp(10, 1, 1, 0.2, 1, 1, 0.2)

In [29]:
Z @ np.linalg.inv(Z.T @ Z)

,0,1
0,0.107013,-0.059665
1,0.043636,0.479554
2,0.120266,-0.172430
3,0.121951,-0.186759
4,0.108884,-0.075584
5,0.117163,-0.146021
6,0.117649,-0.150158
7,0.098000,0.017018
8,0.082711,0.147099
9,0.082729,0.146945


In [ ]:
B = pd.DataFrame(
    [
        draw_b(100,lambda N: linear_dgp(N,1,0,.01,1,1))[1] for i in range(1000)
    ]
)
B.hist(bins=int(np.ceil(np.sqrt(B.shape[0]))))

In [ ]:
# Estimate b_HFUL using the function above. Compare what it spits out to the true beta printed above.
b = HFUL(y, X, Z)
b